In [2]:
import os
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from datetime import datetime, timedelta
def get_start_end(time_1, time_2):
    # 将时间字符串转换为时间间隔
    format_string = '%H:%M:%S.%f'
    time1_obj = datetime.strptime(time_1, format_string)
    time2_obj = datetime.strptime(time_2, format_string)

    # 计算时间间隔的总和
    # total_time = time1_obj - datetime.min + (time2_obj - datetime.min)

    # 计算平均时间间隔
    # average_time = total_time / 2

    # 将平均时间间隔格式化为相同的时间字符串格式
    # average_time_str = (datetime.min + average_time).time().strftime(format_string)
    return time1_obj, time2_obj

In [21]:
def get_duration_time(time_1, time_2):
    # 将时间字符串转换为时间间隔
    format_string = '%H:%M:%S.%f'

    # 计算时间间隔的总和
    total_time = time_2 - time_1

    # 将平均时间间隔格式化为相同的时间字符串格式
    average_time_str = (datetime.min + total_time).time().strftime(format_string)
    return average_time_str

In [22]:
def getTimeAndText(ass_path):
    with open(ass_path, 'r', encoding='utf-16') as file:
        lines = file.readlines()
    keyword = "zhengwen"
    duration_time_list = []
    text_list = []
    start_time_list = []
    end_time_list = []
    for line in lines:
        if line.startswith('Dialogue') and keyword in line:
            content = line.split(":", 1)[1].strip()
            line_list = content.split(",")
            start_time, end_time = get_start_end(line_list[1], line_list[2])
            start_time_list.append(start_time.strftime('%H:%M:%S.%f'))
            end_time_list.append(end_time.strftime('%H:%M:%S.%f'))
            duration_time = get_duration_time(start_time, end_time)
            duration_time_list.append(duration_time)
            text_list.append(line_list[-1])
    return duration_time_list, text_list, start_time_list, end_time_list

In [ ]:
import json
import subprocess

def extract_audio_at_time(video_path, output_path, time_stamp, duration_time):
    ffmpeg_cmd = [
        'ffmpeg',
        '-ss', time_stamp,
        '-i', video_path,
        "-t", duration_time,
        "-q:a", "0",
        output_path
    ]

    subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# for id in [str(x).zfill(2) for x in range(15, 29)]:

for k in range(1,29):
    save_path = f"/content/drive/MyDrive/GPTData/Haruhi_audio/{k:0=2d}/"
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    data = []
    ass_path = f"[CASO][Suzumiya_Haruhi_no_Yuuutsu][{k:0=2d}]"
    video_path = f"[CASO][Suzumiya_Haruhi_no_Yuuutsu][{k:0=2d}]"

    directory = "/content/drive/MyDrive/GPTData/Haruhi_all"
    for filename in os.listdir(directory):
        if filename.endswith("ass") and filename.startswith(ass_path):
            ass_path = filename
        if filename.endswith("mkv") and filename.startswith(video_path):
            video_path = filename

    ass_path = os.path.join(directory, ass_path)
    video_path = os.path.join(directory, video_path)

    duration_time_list, text_list, start_time_list, end_time_list = getTimeAndText(ass_path)
    # print(duration_time_list)
    # print(start_time_list)
    # print(end_time_list)
    for i in range(len(start_time_list)):
        time_stamp = start_time_list[i]
        duration_time = duration_time_list[i]
        output_path = save_path + f'{text_list[i]}{start_time_list[i]}.wav'
        extract_audio_at_time(video_path, output_path, time_stamp, duration_time)
        data.append({"text":str(text_list[i]), "timestamp":str(time_stamp), "path":f"{text_list[i]}{time_stamp}.wav"})

    with open(f'{save_path}/info_{k:0=2d}.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)